In [0]:
import pyspark
from pyspark.sql.types import StructType,StructField, StringType, IntegerType,ArrayType,MapType

data = [
 ("James", "A", "Smith", 3000),
 ("Michael", "B", "Rose", 4000),
 ("Robert", "C", "Williams", 2500),
 ("Maria", "D", "Jones", 5000)
]

schema = StructType([StructField("first_name", StringType(), True),
                     StructField("middle_name", StringType(), True),
                     StructField("surname", StringType(), True),
                     StructField("salary", IntegerType(), True)])

df = spark.createDataFrame(data = data, schema = schema)
df.printSchema()
df.show(truncate = False)

In [0]:
data = [
 ("James", "A", "Smith", "NY", 3000),
 ("Michael", "B", "Rose", "CA", 4000),
 ("Robert", "C", "Williams", "TX", 2500),
 ("Maria", "D", "Jones", "CA", 5000),
 ("Jen", "", "Brown", "NY", 0)
]

cols = ["first", "middle", "last", "state", "salary"]

df = spark.createDataFrame(data, cols)


In [0]:
df2 = df.select("first","last","state")
df2.show()
from pyspark.sql.functions import col
df3 = df.filter(col("salary")>3000)
df3.show()

df4 = df.filter((col("state") == "CA") | (col("state") == "NY"))
df4.show()

In [0]:
#Create a new column salary_k = salary / 1000
df1 = df.withColumn("salary_k",col("salary")/1000).show(truncate=False)

#Create a new column full_name = first + " " + last
from pyspark.sql.functions import concat, lit, col

df_1 =df.withColumn("full_name", concat(col("first"), lit(" "), col("last")))
df_1.show()

In [0]:
#sort by salary descending
df.sort(col("salary").desc()).show(truncate=False)
#Group by state and find the average salary

df_sorted = df.orderBy(col("salary").desc())
df_top_per_state = df_sorted.dropDuplicates(["state"])
df_top_per_state.show(truncate=False)

In [0]:
df.groupBy("state").avg("salary").show(truncate=False)

#total_employers
df.groupBy("state").count().show(truncate=False)


In [0]:
state_dim = [
  ("NY", "East Coast"),
  ("CA", "West Coast"),
  ("TX", "South Central")
]

df_dim = spark.createDataFrame(state_dim, ["state", "region"])


In [0]:
df_inner = df.join(df_dim, df.state == df_dim.state, "inner")
df_inner.show(truncate=False)
df_left = df.join(df_dim, df.state == df_dim.state, "left")
df_left.show(truncate=False)
df_leftan = df.join(df_dim, df.state == df_dim.state, "leftanti")
df_leftan.show(truncate=False)
print("Unmatched count:", df_leftan.count())


In [0]:
from pyspark.sql.types import *
from pyspark.sql import Row

orders_data = [
    ("O1", "C1", "2024-01-10", 1200.0, "COMPLETED", '{"payment_mode":"CARD","device":"MOBILE","promo_code":"NEW10"}'),
    ("O2", "C1", "2024-01-15", 800.0, "COMPLETED", '{"payment_mode":"UPI","device":"WEB","promo_code":null}'),
    ("O3", "C1", "2024-01-20", 0.0, "CANCELLED", '{"payment_mode":"CARD","device":"MOBILE","promo_code":"NEW10"}'),

    ("O4", "C2", "2024-02-05", 500.0, "COMPLETED", '{"payment_mode":"COD","device":"MOBILE","promo_code":null}'),
    ("O5", "C2", "2024-02-18", -100.0, "COMPLETED", '{"payment_mode":"UPI","device":"WEB","promo_code":"SAVE5"}'),

    ("O6", "C3", "2024-03-01", 2000.0, "COMPLETED", '{"payment_mode":"CARD","device":"WEB","promo_code":"BIGSALE"}'),

    # Customer not present in customers table
    ("O7", "C4", "2024-03-10", 1500.0, "COMPLETED", '{"payment_mode":"CARD","device":"MOBILE","promo_code":"NEW10"}')
]

orders_schema = StructType([
    StructField("order_id", StringType(), False),
    StructField("customer_id", StringType(), False),
    StructField("order_date", StringType(), True),
    StructField("order_amount", DoubleType(), True),
    StructField("order_status", StringType(), True),
    StructField("order_metadata", StringType(), True)
])

orders_df = spark.createDataFrame(orders_data, orders_schema)
orders_df.show(truncate=False)



In [0]:
customers_data = [
    ("C1", "Alice Johnson", "2023-12-01", "USA"),
    ("C2", "Bob Smith", "2023-11-15", "India"),
    ("C3", "Charlie Brown", "2024-01-05", "UK")
]

customers_schema = StructType([
    StructField("customer_id", StringType(), False),
    StructField("customer_name", StringType(), True),
    StructField("signup_date", StringType(), True),
    StructField("country", StringType(), True)
])

customers_df = spark.createDataFrame(customers_data, customers_schema)
customers_df.show(truncate=False)


In [0]:
#Task1.2
from pyspark.sql.functions import *

orders_df_1 = orders_df.withColumn("order_date",
to_date(col("order_date"),"yyyy-MM-dd").alias("order_date"))



orders_df_1.printSchema()


customers_df_1 = customers_df.withColumn(("signup_date"),to_date(col("signup_date"),"yyyy-MM-dd").alias("signup_date"))
customers_df_1.show()
customers_df_1.printSchema()

In [0]:
orders_df_2 = orders_df_1 \
    .withColumn("payment_mode", json_tuple(col("order_metadata"), "payment_mode")) \
    .withColumn("device", json_tuple(col("order_metadata"), "device")) \
    .withColumn("promo_code", json_tuple(col("order_metadata"), "promo_code"))

orders_df_2.show(truncate=False)
orders_df_2.printSchema()


In [0]:
order_df_up = orders_df_2.filter((col("order_status") == "COMPLETED") & (col("order_amount") > 0))
order_df_up.show(truncate=False)


In [0]:
Ord_cus = order_df_up.join(customers_df_1, order_df_up.customer_id == customers_df_1.customer_id, "left")
#order_df_up.join(customers_df_1, on="customer_id", how="left")
Ord_cus.show(truncate=False)

In [0]:
from pyspark.sql.functions import sum, count, avg

Ord_cus_clean = Ord_cus.drop(customers_df_1.customer_id)

cust_metrics = (
    Ord_cus_clean
    .groupBy("customer_id")
    .agg(
        sum("order_amount").alias("total_order_amount"),
        count("order_id").alias("total_orders"),
        avg("order_amount").alias("average_order_value")
    )
)

cust_metrics.show(truncate=False)


In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number
windowSpec  = Window.partitionBy("customer_id").orderBy(col("order_date").desc())

Ord_cus_clean2 = Ord_cus_clean.withColumn("row_number",row_number().over(windowSpec))

Ord_cus_clean3 = Ord_cus_clean2.filter(
    col("row_number") == 1
)
display(Ord_cus_clean3)

In [0]:
Ord_cus_flagged = Ord_cus_clean2.withColumn(
    "is_latest_order",
    when(col("row_number") == 1, True).otherwise(False)
)

Ord_cus_flagged.show(truncate=False)


In [0]:

#customers_df_1
#cust_metrics

final_cust = customers_df_1.join(cust_metrics , on="customer_id",how="left")
final_cust = final_cust.drop("signup_date")
final_cust.show(truncate= False)



In [0]:

#display(Ord_cus_clean3)

column_need = ["customer_id","order_date","payment_mode","device"]
order_last = Ord_cus_clean3.select(column_need)
order_last.show(truncate = False)

In [0]:
final_df  = final_cust.join(order_last , on="customer_id",how="left")
final_df = final_df.withColumnRenamed("order_date","last_order")
final_df.show(truncate = False)